In [113]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from numpy.random import Generator, PCG64

import tqdm
import json
import inspect
import warnings
from IPython.display import display, Markdown, Latex

warnings.filterwarnings('ignore')

In [6]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

##### Config

In [118]:
pd.set_option("display.max_columns", None)

In [215]:
def get_sample_from_distribution(n_sample, random_state, params):
    """
    Return a sample of size N from de distribution
    """
    scale, loc, b, a = params["scale"], params["loc"], params["b"], params["a"]
    #logger.info(f"param a: {a} \n param b: {b} \n param loc: {loc} \n param scale: {scale}")
    distribution = stats.johnsonsu(a=a, b=b, loc=loc, scale=scale)
    #distribution.random_state = random_state
    return distribution.rvs(size=n_sample) if not pd.isnull(n_sample) else 0

# Data
## Daily Demand

In [7]:
raw = pd.read_csv("../../data/output_layer_process.csv")
logger.info(f'Quantity of rows input {len(raw.index)}')

INFO:__main__:Quantity of rows input 71774


In [9]:
raw.head()

,year,month,cod_customer,lat,lon,demand,n_dates,demand_daily,layer,pixel
0,2022,8,109895,-16.511478,-68.097017,1.00,2,1,0,168.0
1,2022,6,810527,-16.488097,-68.155177,1.56,3,1,0,209.0
2,2020,12,690147,-16.524460,-68.077230,1.56,3,1,0,154.0
3,2020,12,690146,-16.503893,-68.119619,1.56,5,1,0,181.0
4,2021,10,42876,-16.544710,-68.043090,1.56,5,1,0,125.0


In [8]:
pd.pivot_table(data=raw, index=["year","month"], columns=["layer"], values="cod_customer", aggfunc="count")

layer            0     1      2
year month                     
2020 1      1421.0   9.0   42.0
     2      1253.0  14.0   46.0
     3      1057.0   4.0   29.0
     4       962.0   6.0   18.0
     5      1138.0   6.0   31.0
     6      1284.0   9.0   48.0
     7      1293.0  14.0   49.0
     8      1189.0   5.0   38.0
     9      1525.0   5.0   56.0
     10     1787.0   7.0   73.0
     11     1685.0  17.0   75.0
     12     2030.0  17.0   93.0
2021 1      1391.0   7.0   82.0
     2       385.0   NaN    9.0
     3      1734.0   3.0   68.0
     4      1683.0   5.0   69.0
     5      1530.0  13.0   64.0
     6      1473.0  11.0   46.0
     7      1732.0  12.0   75.0
     8      1795.0  13.0   71.0
     9      2055.0  13.0   87.0
     10     2339.0  20.0   91.0
     11     1896.0  15.0   97.0
     12     2513.0  26.0  112.0
2022 1      1760.0  17.0   83.0
     2      2495.0  18.0   99.0
     3      2594.0  13.0  107.0
     4      2459.0  13.0   86.0
     5      2214.0  20.0   86.0
     6      2831.0  15.0  127.0
     7      2558.0  24.0   93.0
     8      2534.0  22.0   92.0
     9      2758.0  22.0  128.0
     10     2998.0  26.0  134.0
     11     2853.0  29.0  122.0
     12     3293.0  37.0  144.0

## Params Distribution

In [18]:
f = open('../../data/scenario-generation/distribution_params.json')
params = json.load(f)
params

{'low': {'1': {'a': -4.807745279338757,
   'b': 0.773388884111248,
   'loc': 0.9927181072134932,
   'scale': 0.003381593167730684},
  '2': {'a': -4.816430790641132,
   'b': 0.7797548561312492,
   'loc': 0.99437811936145,
   'scale': 0.0036337374577629688},
  '3': {'a': -4.947549766877758,
   'b': 0.7777076520263277,
   'loc': 0.9911220946881644,
   'scale': 0.003033144998833001},
  '4': {'a': -4.919360522179474,
   'b': 0.7740023485024676,
   'loc': 0.9909736001278993,
   'scale': 0.0030391021166006225},
  '5': {'a': -4.126886682947598,
   'b': 0.7850403991916988,
   'loc': 0.9882063380291453,
   'scale': 0.00926217123085547},
  '6': {'a': -4.890860326168944,
   'b': 0.7825434846293198,
   'loc': 0.9848989893866555,
   'scale': 0.0037570725613611314},
  '7': {'a': -5.081622394812156,
   'b': 0.787335957455368,
   'loc': 0.9840140799725035,
   'scale': 0.002843352997077343},
  '8': {'a': -5.052541315493812,
   'b': 0.7838769849244744,
   'loc': 0.98280708415095,
   'scale': 0.0028468902

## Customer per Pixel-Month related to 2022

In [189]:
display(Markdown(f"Considerando el año 2022, vamos a samplear tantas veces como clientes tengamos en cada pixel, por cada pixel se sampleara size(pixel) * 12 meses"))
pivot_tbl = pd.pivot_table(data=raw[(raw.year==2022)], index=["pixel"], columns=["layer"], values="cod_customer", aggfunc="nunique")
display(pivot_tbl.T)

Considerando el año 2022, vamos a samplear tantas veces como clientes tengamos en cada pixel, por cada pixel se sampleara size(pixel) * 12 meses

pixel,26.0,42.0,71.0,72.0,73.0,87.0,88.0,104.0,105.0,106.0,107.0,108.0,109.0,111.0,115.0,116.0,119.0,120.0,121.0,122.0,123.0,124.0,125.0,126.0,127.0,130.0,131.0,132.0,133.0,134.0,135.0,136.0,137.0,138.0,139.0,140.0,141.0,142.0,143.0,146.0,147.0,148.0,149.0,150.0,151.0,152.0,153.0,154.0,155.0,156.0,157.0,158.0,161.0,162.0,163.0,164.0,165.0,166.0,167.0,168.0,169.0,170.0,171.0,172.0,173.0,177.0,178.0,179.0,180.0,181.0,182.0,183.0,184.0,185.0,188.0,189.0,192.0,193.0,194.0,195.0,196.0,197.0,198.0,199.0,200.0,201.0,208.0,209.0,210.0,211.0,212.0,213.0,214.0,215.0,216.0,217.0,218.0,224.0,226.0,227.0,228.0,229.0,230.0,234.0,235.0,241.0,242.0,244.0,245.0,246.0,258.0,259.0,261.0,262.0,263.0,274.0,275.0,279.0,280.0,290.0,291.0
layer,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,3.0,3.0,12.0,25.0,36.0,10.0,7.0,6.0,1.0,8.0,1.0,11.0,1.0,2.0,11.0,3.0,6.0,59.0,77.0,74.0,43.0,49.0,54.0,22.0,30.0,34.0,56.0,36.0,2.0,1.0,35.0,54.0,21.0,60.0,1.0,5.0,62.0,63.0,11.0,39.0,44.0,12.0,45.0,40.0,47.0,40.0,40.0,21.0,3.0,12.0,8.0,9.0,2.0,26.0,165.0,96.0,40.0,45.0,18.0,20.0,50.0,8.0,7.0,27.0,1.0,25.0,128.0,226.0,171.0,84.0,98.0,81.0,42.0,28.0,4.0,32.0,10.0,96.0,184.0,335.0,132.0,120.0,104.0,65.0,35.0,18.0,57.0,78.0,88.0,109.0,85.0,96.0,66.0,40.0,35.0,6.0,4.0,14.0,48.0,60.0,46.0,117.0,20.0,3.0,1.0,3.0,58.0,4.0,68.0,86.0,30.0,6.0,11.0,25.0,14.0,34.0,3.0,1.0,11.0,12.0,3.0
1,NaN,2.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,3.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0,1.0,NaN,NaN,3.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,3.0,3.0,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,2.0,2.0,NaN,1.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,NaN,2.0,1.0,1.0,3.0,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN,1.0,7.0,5.0,4.0,4.0,NaN,NaN,4.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,3.0,5.0,NaN,NaN,6.0,4.0,NaN,6.0,1.0,1.0,2.0,1.0,4.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,1.0,2.0,4.0,5.0,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,9.0,7.0,15.0,7.0,4.0,3.0,3.0,1.0,NaN,1.0,2.0,8.0,22.0,41.0,19.0,5.0,7.0,6.0,1.0,NaN,3.0,4.0,4.0,7.0,1.0,4.0,3.0,NaN,1.0,1.0,1.0,NaN,3.0,2.0,1.0,7.0,NaN,NaN,NaN,NaN,2.0,NaN,4.0,5.0,NaN,NaN,NaN,NaN,1.0,4.0,NaN,NaN,1.0,NaN,NaN


## A) Layer Low

In [192]:
PERIODS = [i for i in range(1,13)]
LAYER="low"

In [226]:
df_low = pd.DataFrame(pivot_tbl[0]).reset_index()
df_low.rename(columns={
        0: "n_customers",
    }, inplace=True
)
df_low["layer"] = LAYER
for period in PERIODS:
    logger.info(f'[PERIOD] {period}')
    # set RANDOM STATE
    random_state = Generator(PCG64(12345))
    # sampling by pixel
    df_low[period] = df_low.apply(lambda x: sum(
                                        get_sample_from_distribution(
                                            n_sample = int(x["n_customers"]), 
                                            random_state = random_state,
                                            params = params[LAYER][str(period)],
                                        )
                                    ),
                                    axis=1
                         )
df_low["average"] = df_low.apply(lambda x: np.mean([x[i] for i in PERIODS]), axis=1)

INFO:__main__:[PERIOD] 1
INFO:__main__:[PERIOD] 2
INFO:__main__:[PERIOD] 3
INFO:__main__:[PERIOD] 4
INFO:__main__:[PERIOD] 5
INFO:__main__:[PERIOD] 6
INFO:__main__:[PERIOD] 7
INFO:__main__:[PERIOD] 8
INFO:__main__:[PERIOD] 9
INFO:__main__:[PERIOD] 10
INFO:__main__:[PERIOD] 11
INFO:__main__:[PERIOD] 12


In [227]:
df_low

,pixel,n_customers,layer,1,2,3,4,5,6,7,8,9,10,11,12,average
0,26.0,3.0,medium,124.520526,125.208701,193.093588,59.286188,69.566770,83.148493,90.916283,93.339106,73.610741,82.386627,81.186988,86.550922,96.901244
1,42.0,3.0,medium,68.933471,59.592073,86.905103,100.746086,62.269311,107.150655,72.108350,350.049324,123.711595,85.899255,58.070654,134.032546,109.122369
2,71.0,12.0,medium,310.034708,364.378066,318.776298,335.185523,350.322880,426.675397,360.648425,307.908644,275.849215,352.935654,350.174104,498.264053,354.262747
3,72.0,25.0,medium,832.260283,671.756198,627.545912,690.302071,650.574645,743.830308,822.308018,697.273241,883.221793,741.815082,635.678536,769.300973,730.488922
4,73.0,36.0,medium,1050.029025,1031.508105,1036.337079,1032.404167,1030.133335,1078.815707,952.305560,1235.424524,1068.832081,1140.848575,995.453648,1137.608781,1065.808382
5,87.0,10.0,medium,249.211837,281.038430,364.663725,301.825655,283.575875,355.369572,290.598101,357.203137,274.155948,296.286662,243.491229,293.183339,299.216959
6,88.0,7.0,medium,196.114700,166.452570,164.686119,154.788731,247.939744,148.267402,248.684970,157.701507,203.240081,184.104413,164.684260,261.802678,191.538931
7,104.0,6.0,medium,210.800189,170.753472,175.923151,210.557869,204.328994,260.032591,158.934487,115.176077,139.692234,210.701181,152.522124,151.862494,180.107072
8,105.0,1.0,medium,40.908766,23.498133,80.032400,24.497133,41.225694,17.222646,47.393124,17.790289,46.981507,30.252153,27.965050,21.445174,34.934339
9,106.0,8.0,medium,231.694865,201.851036,249.817112,255.675991,200.776720,243.803939,213.368453,214.027437,230.401530,297.318934,227.921977,285.188781,237.653898


## B) Layer Medium

In [208]:
PERIODS = [i for i in range(1,13)]
LAYER="medium"

In [230]:
df_medium = pd.DataFrame(pivot_tbl[2]).reset_index()
df_medium.rename(columns={
        2: "n_customers",
    }, inplace=True
)
df_medium = df_medium.dropna()
df_medium["layer"] = LAYER
for period in PERIODS:
    logger.info(f'[PERIOD] {period}')
    # set RANDOM STATE
    random_state = Generator(PCG64(12345))
    # sampling by pixel
    df_medium[period] = df_medium.apply(lambda x: sum(
                                        get_sample_from_distribution(
                                            n_sample = int(x["n_customers"]), 
                                            random_state = random_state,
                                            params = params[LAYER][str(period)],
                                        )
                                    ),
                                    axis=1
                         )
df_medium["average"] = df_medium.apply(lambda x: np.mean([x[i] for i in PERIODS]), axis=1)

INFO:__main__:[PERIOD] 1
INFO:__main__:[PERIOD] 2
INFO:__main__:[PERIOD] 3
INFO:__main__:[PERIOD] 4
INFO:__main__:[PERIOD] 5
INFO:__main__:[PERIOD] 6
INFO:__main__:[PERIOD] 7
INFO:__main__:[PERIOD] 8
INFO:__main__:[PERIOD] 9
INFO:__main__:[PERIOD] 10
INFO:__main__:[PERIOD] 11
INFO:__main__:[PERIOD] 12


In [231]:
df_medium

,pixel,n_customers,layer,1,2,3,4,5,6,7,8,9,10,11,12,average
1,42.0,2.0,medium,60.450729,50.486929,54.490774,55.938235,46.749768,86.300081,51.078420,70.844765,39.225170,38.651603,60.098567,52.495803,55.567570
2,71.0,1.0,medium,31.116767,21.638890,20.353398,86.245291,66.452647,44.497780,29.276069,21.599468,31.435731,37.280786,19.655230,16.349375,35.491786
3,72.0,1.0,medium,22.446334,41.415481,17.330687,22.672849,23.580586,22.344874,22.658912,21.955000,33.864395,29.201422,36.015771,18.914524,26.033403
4,73.0,3.0,medium,84.903100,66.328509,148.679095,83.886579,77.497614,61.228273,57.097859,60.265885,83.414300,119.035098,53.093593,58.136372,79.463856
6,88.0,1.0,medium,74.485362,19.811770,16.076008,34.947927,24.295054,17.394202,207.808510,23.331675,16.734923,21.485316,30.217888,22.305873,42.407876
7,104.0,1.0,medium,19.337453,49.791862,22.297671,20.224379,17.945334,35.444638,22.428875,20.292146,17.997634,19.481738,24.684357,128.911498,33.236465
9,106.0,1.0,medium,46.989966,17.848494,29.371970,30.435647,18.604084,22.859188,24.962837,16.928731,22.788175,16.765831,17.500915,20.739604,23.816287
13,111.0,1.0,medium,34.053534,30.449088,22.792991,29.682700,32.083113,21.077721,20.132351,16.996617,18.000107,36.237870,23.450861,30.615409,26.297697
14,115.0,1.0,medium,39.632405,19.793151,31.500488,40.495263,22.328263,16.862295,25.116088,18.438689,26.082941,29.357996,110.407407,52.087993,36.008581
15,116.0,1.0,medium,41.140945,20.684739,25.249798,17.220654,46.882157,24.498188,17.988080,16.374622,25.591601,20.746480,18.702847,26.109433,25.099129


## C) Layer high

In [232]:
PERIODS = [i for i in range(1,13)]
LAYER="high"

In [233]:
df_high = pd.DataFrame(pivot_tbl[1]).reset_index()
df_high.rename(columns={
        1: "n_customers",
    }, inplace=True
)
df_high = df_high.dropna()
df_high["layer"] = LAYER
for period in PERIODS:
    logger.info(f'[PERIOD] {period}')
    # set RANDOM STATE
    random_state = Generator(PCG64(12345))
    # sampling by pixel
    df_high[period] = df_high.apply(lambda x: sum(
                                        get_sample_from_distribution(
                                            n_sample = int(x["n_customers"]), 
                                            random_state = random_state,
                                            params = params[LAYER][str(period)],
                                        )
                                    ),
                                    axis=1
                         )
df_high["average"] = df_high.apply(lambda x: np.mean([x[i] for i in PERIODS]), axis=1)

INFO:__main__:[PERIOD] 1
INFO:__main__:[PERIOD] 2
INFO:__main__:[PERIOD] 3
INFO:__main__:[PERIOD] 4
INFO:__main__:[PERIOD] 5
INFO:__main__:[PERIOD] 6
INFO:__main__:[PERIOD] 7
INFO:__main__:[PERIOD] 8
INFO:__main__:[PERIOD] 9
INFO:__main__:[PERIOD] 10
INFO:__main__:[PERIOD] 11
INFO:__main__:[PERIOD] 12


In [234]:
df_high

,pixel,n_customers,layer,1,2,3,4,5,6,7,8,9,10,11,12,average
1,42.0,2.0,high,159.580537,174.437481,152.443906,149.246172,183.019419,139.417681,192.477997,134.829965,131.564816,138.636530,151.782949,255.080734,163.543182
4,73.0,2.0,high,162.224538,135.496453,175.402607,129.721937,233.954506,175.086731,183.282810,206.491907,169.441624,185.696162,245.481093,213.532962,184.651111
17,120.0,1.0,high,85.196787,84.684008,61.354431,68.096763,105.270393,60.330011,62.399788,148.186874,119.986700,117.556660,80.171552,115.476665,92.392553
18,121.0,2.0,high,229.107643,165.468866,128.110651,153.899876,155.070709,157.039843,337.458911,217.490546,146.971157,163.894811,130.233533,143.840336,177.382240
19,122.0,3.0,high,282.971120,411.800850,288.996993,260.096703,208.897387,335.976059,205.444321,271.142308,327.240572,227.336231,304.749135,258.852846,281.958710
24,127.0,1.0,high,71.383434,69.756802,102.106685,91.965553,98.496413,96.215054,72.772517,101.030705,71.778437,77.906618,85.321978,114.692116,87.785526
30,135.0,2.0,high,289.482894,152.199500,125.650406,233.832176,154.317572,171.127853,176.676021,132.680281,190.814262,129.065651,252.292875,143.487767,179.302272
32,137.0,1.0,high,86.246277,80.647315,80.543926,103.471591,67.408427,108.094651,71.146863,133.374025,165.177051,140.265261,65.896089,97.601832,99.989442
33,138.0,1.0,high,76.851163,66.870286,440.298055,66.512167,89.451079,72.269497,74.888970,64.657830,60.132129,69.867042,71.944391,101.845425,104.632336
36,141.0,3.0,high,315.059263,248.142688,369.535750,255.052535,256.489681,234.358007,301.634057,227.327709,238.603683,218.144623,291.451989,365.823597,276.801965


## Merge Data

In [237]:
df_output = pd.concat([
    df_low,
    df_medium,
    df_high,
    ]
).reset_index(drop=True)

In [240]:
df_output["id"] = df_output.apply(lambda x: x["layer"]+"-"+str(int(x["pixel"])), axis=1)
df_output

,pixel,n_customers,layer,1,2,3,4,5,6,7,8,9,10,11,12,average,id
0,26.0,3.0,medium,124.520526,125.208701,193.093588,59.286188,69.566770,83.148493,90.916283,93.339106,73.610741,82.386627,81.186988,86.550922,96.901244,medium-26
1,42.0,3.0,medium,68.933471,59.592073,86.905103,100.746086,62.269311,107.150655,72.108350,350.049324,123.711595,85.899255,58.070654,134.032546,109.122369,medium-42
2,71.0,12.0,medium,310.034708,364.378066,318.776298,335.185523,350.322880,426.675397,360.648425,307.908644,275.849215,352.935654,350.174104,498.264053,354.262747,medium-71
3,72.0,25.0,medium,832.260283,671.756198,627.545912,690.302071,650.574645,743.830308,822.308018,697.273241,883.221793,741.815082,635.678536,769.300973,730.488922,medium-72
4,73.0,36.0,medium,1050.029025,1031.508105,1036.337079,1032.404167,1030.133335,1078.815707,952.305560,1235.424524,1068.832081,1140.848575,995.453648,1137.608781,1065.808382,medium-73
5,87.0,10.0,medium,249.211837,281.038430,364.663725,301.825655,283.575875,355.369572,290.598101,357.203137,274.155948,296.286662,243.491229,293.183339,299.216959,medium-87
6,88.0,7.0,medium,196.114700,166.452570,164.686119,154.788731,247.939744,148.267402,248.684970,157.701507,203.240081,184.104413,164.684260,261.802678,191.538931,medium-88
7,104.0,6.0,medium,210.800189,170.753472,175.923151,210.557869,204.328994,260.032591,158.934487,115.176077,139.692234,210.701181,152.522124,151.862494,180.107072,medium-104
8,105.0,1.0,medium,40.908766,23.498133,80.032400,24.497133,41.225694,17.222646,47.393124,17.790289,46.981507,30.252153,27.965050,21.445174,34.934339,medium-105
9,106.0,8.0,medium,231.694865,201.851036,249.817112,255.675991,200.776720,243.803939,213.368453,214.027437,230.401530,297.318934,227.921977,285.188781,237.653898,medium-106


In [241]:
df_output.to_csv('../../data/scenarios.csv', index=False)